# időjárárás

https://odp.met.hu/tree.html

historikus adat:
https://odp.met.hu/climate/station_data_series/daily/from_1901/
## Letöltés opd-ről:

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import zipfile
from config import generated_files_path
from utils import create_folder_if_not_exists

url_base= "https://odp.met.hu/climate/station_data_series/daily/from_1901/"
urls=["maximum_temperature","mean_temperature","minimum_temperature","precipitation_sum","meta"]
data_urls=["maximum_temperature","mean_temperature","minimum_temperature","precipitation_sum"]

def get_soup(url):
    return bs(requests.get(url).text, 'html.parser')
filetype = ".zip"

datafolder_base = generated_files_path+"/odp/"

import os


create_folder_if_not_exists(datafolder_base)


def  scrap_and_load_input_data(t):
    url=url_base+t+"/"
    # dfs: list of all the new dataframes that comes from the new scrap
    dfs = []
    # check what files are on the page
    found_links=[]
    data_path=datafolder_base+t+"/"
    create_folder_if_not_exists(data_path)

    for link in get_soup(url).find_all('a'):
        file_link = link.get('href')
        # links on icon too (?)
        if file_link in found_links:
            continue
        found_links.append(file_link)
        # print(link)
        if t=='meta' and file_link.endswith('.csv'):
            print('Downloading meta file: {} '.format(file_link))
        elif filetype not in file_link:
            print('Not searched file {}'.format(file_link))
            continue
        # get the upload time of the file
        #d = get_datetime(file_link)
        #print('File found for {}'.format(d))
        # if we already loaded the file into db, or not the expected format then skip
        #if d in dates:
        #    print('Already loaded {}, SKIPPED'.format(d))
        #    continue
        #if not d:
        #    print('Not per date measurement file, SKIPPED')
        #    continue
        print('Downloading {}'.format(file_link))

        # file with data not loaded into weather table yet download
        with open(data_path + file_link, 'wb') as file:
            response = requests.get(url + file_link)
            print('Response Code:', response.status_code)
            file.write(response.content)


for t in urls:
    scrap_and_load_input_data(t)


Directory 'data/generated_files//odp/' already exists.
Directory 'data/generated_files//odp/maximum_temperature/' already exists.
Not searched file https://odp.met.hu
Not searched file ?C=N;O=D
Not searched file ?C=M;O=A
Not searched file ?C=S;O=A
Not searched file ?C=D;O=A
Not searched file /climate/station_data_series/daily/from_1901/
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Directory 'data/generated_files//odp/mean_temperature/' already exists.
Not searched file https://odp.met.hu
Not searched file ?C=N;O=D
Not searched file ?C=M;O=A
Not searched file ?C=S;O=A
Not searched file ?C=D;O=A
Not searched file /climate/station_data_series/daily/from_1901/
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200
Response Code: 200


In [9]:
import pandas as pd
data_urls=["maximum_temperature","mean_temperature","minimum_temperature","precipitation_sum"]

def extract_and_assemble(t):
    print(t)
    res_l=[]
    data_path=datafolder_base+t+"/"
    for fn in os.listdir(data_path):
        if not fn.endswith(".zip"):
            continue
        create_folder_if_not_exists(data_path+"extracted/")
        
        extracted_folder = data_path+"extracted/" + fn.split('.')[0] + '/'
        #print(extracted_folder)
        #print(f'Extracting {data_path+"extracted/" + fn} ')
        print(fn)
        loc_name=fn.split('_')[2]
        
        # create folder with the same name of zip, and extract content in it
        with zipfile.ZipFile(data_path + fn, 'r') as zip_ref:
            if not os.path.exists(extracted_folder):
                os.mkdir(extracted_folder)
            zip_ref.extractall(extracted_folder)
            # iterate over all the downloaded files (per zip, with hourly update we expect a single file)
            for f in os.listdir(extracted_folder):
                print('reading ',f)
                # content of file to df
                df = pd.read_csv(extracted_folder + f, sep=";", skipinitialspace=True,parse_dates=['Time'], date_format='%Y%m%d')
                df['Loc']=loc_name
                df.set_index(['Loc','Time'], inplace=True)
                df.columns = df.columns.str.strip()
                df.drop(columns='EOR', inplace=True)
                #clean_weather_df(df)
                # collect dataframes in order to process for the result fusarium table
                #print(df.head())
                res_l.append(df)
                #print(len(res_l))
                # load the data from file into weather table
                #append_to_table(data_table_name, df, engine)
                # delete file
                os.remove(extracted_folder + f)
            # delete directory
            os.rmdir(extracted_folder)
    return pd.concat(res_l)
        #os.remove(hourly_data_folder + file_link)
        # push the file update time into  update_time table
        #days_df = pd.DataFrame.from_dict({'Time': [pd.to_datetime(d)]})
        #append_to_table(day_table_name, days_df, engine)
    #ret=pd.DataFrame()
    #if len(dfs)>0:
    #    ret= pd.concat(dfs,axis=0, ignore_index=True)
    #return ret
l=[extract_and_assemble(t) for t in data_urls]

res = pd.concat(l,axis=1)

        

maximum_temperature
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Budapest_19012023.csv.zip
reading  tx_o_Budapest_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Debrecen_19012023.csv.zip
reading  tx_o_Debrecen_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Keszthely_19012023.csv.zip
reading  tx_o_Keszthely_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Miskolc_19012023.csv.zip
reading  tx_o_Miskolc_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Nyiregyhaza_19012023.csv.zip
reading  tx_o_Nyiregyhaza_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' already exists.
tx_o_Pecs_19012023.csv.zip
reading  tx_o_Pecs_19012023.csv
Directory 'data/generated_files//odp/maximum_temperature/extracted/' a

In [11]:
res.to_csv(datafolder_base+ "weather.csv")

In [6]:
def assemble_meta(t):
    data_path=datafolder_base+t+"/"
    df_s=[]
    for fn in os.listdir(data_path):
        loc_name=fn.split('_')[2]
        fn=data_path+fn     
        print("Reading {}".format(fn))
        df = pd.read_csv(fn, sep=";", skipinitialspace=True,parse_dates=['StartDate   ','EndDate     '], date_format='%Y%m%d%H%M')
        df['Loc']=loc_name
        #df.set_index(['Loc','Time'], inplace=True)
        df.columns = df.columns.str.strip()
        df.drop(columns='EOR', inplace=True)
        df_s.append(df)
    return pd.concat(df_s)   
weather_meta=assemble_meta("meta")

        

Reading data/generated_files//odp/meta/r_meta_Budapest_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Debrecen_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Keszthely_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Miskolc_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Nyiregyhaza_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Pecs_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Sopron_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Szeged_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Szombathely_19012023.csv
Reading data/generated_files//odp/meta/r_meta_Turkeve_19012023.csv
Reading data/generated_files//odp/meta/t_meta_Budapest_19012023.csv
Reading data/generated_files//odp/meta/t_meta_Debrecen_19012023.csv
Reading data/generated_files//odp/meta/t_meta_Keszthely_19012023.csv
Reading data/generated_files//odp/meta/t_meta_Miskolc_19012023.csv
Reading data/generated_files//odp/meta/t_meta_Nyire

In [34]:
weather_meta.head()

,StationNumber,StartDate,EndDate,Latitude,Longitude,Elevation,StationName,Loc
0,44119,1900-02-01,1910-02-28 23:59:00,47.4994,19.0400,120.0,Budapest Víziváros,Budapest
1,44120,1910-03-01,1985-03-31 23:59:00,47.5114,19.0261,118.0,Budapest Országút,Budapest
2,44121,1985-04-01,2024-04-09 00:00:00,47.5111,19.0281,152.3,Budapest belterület,Budapest
0,64304,1868-01-01,1912-06-30 23:59:00,47.5500,21.6167,123.0,Debrecen Pallag,Debrecen
1,64305,1912-07-01,1944-08-31 23:59:00,47.5956,21.6453,129.0,Debrecen Pallag,Debrecen


In [35]:
weather_meta.to_csv(datafolder_base+"weather_meta_full.csv")

In [36]:
def assemble_meta_avg_loc(t):
    data_path=datafolder_base+t+"/"
    df_s=[]
    for fn in os.listdir(data_path):
        loc_name=fn.split('_')[2]
        fn=data_path+fn     
        print("Reading {}".format(fn))
        df = pd.read_csv(fn, sep=";", skipinitialspace=True,parse_dates=['StartDate   ','EndDate     '], date_format='%Y%m%d%H%M')
        ret={}
        print(loc_name)
        ret['Loc']=[loc_name]
        ret['Lat']=[df['Latitude'].mean()]
        ret['Lon']=[df['Longitude'].mean()]
      
        df=pd.DataFrame(ret)
        df_s.append(df)
    return pd.concat(df_s)   
weather_meta_evg=assemble_meta_avg_loc("meta")
weather_meta_evg=weather_meta_evg.set_index('Loc')


Reading data/generated_files//odp/meta/r_meta_Budapest_19012023.csv
Budapest
Reading data/generated_files//odp/meta/r_meta_Debrecen_19012023.csv
Debrecen
Reading data/generated_files//odp/meta/r_meta_Keszthely_19012023.csv
Keszthely
Reading data/generated_files//odp/meta/r_meta_Miskolc_19012023.csv
Miskolc
Reading data/generated_files//odp/meta/r_meta_Nyiregyhaza_19012023.csv
Nyiregyhaza
Reading data/generated_files//odp/meta/r_meta_Pecs_19012023.csv
Pecs
Reading data/generated_files//odp/meta/r_meta_Sopron_19012023.csv
Sopron
Reading data/generated_files//odp/meta/r_meta_Szeged_19012023.csv
Szeged
Reading data/generated_files//odp/meta/r_meta_Szombathely_19012023.csv
Szombathely
Reading data/generated_files//odp/meta/r_meta_Turkeve_19012023.csv
Turkeve
Reading data/generated_files//odp/meta/t_meta_Budapest_19012023.csv
Budapest
Reading data/generated_files//odp/meta/t_meta_Debrecen_19012023.csv
Debrecen
Reading data/generated_files//odp/meta/t_meta_Keszthely_19012023.csv
Keszthely
Rea

In [37]:
weather_meta_evg.to_csv(datafolder_base+"weather_meta_avg.csv")

# Állomások lokációi és a legközelebbi meteorológia mérőállomás


In [1]:
import os

import pandas as pd

In [38]:
from config import data_root,weather_folder,gtfs_folder,gtfs_stops_file,passanger_info_folder,years_to_process
'''data_root="data/"
weather_folder=data_root+"odp/"
gtfs_folder=data_root+"gtfsMavMenetrend/"
gtfs_stops_file="stops.txt"
'''
def get_location_data():
    gtfs_stops=pd.read_csv(gtfs_folder+gtfs_stops_file)
    locs=[]
    passanger_info_folders=[passanger_info_folder+str(y)+"/" for y in years_to_process]
    for l in passanger_info_folders:
        passanger_info_locations_file=l+"t_szolg_helyek.txt"
        places_=pd.read_csv(passanger_info_locations_file,sep=',',encoding='iso-8859-2')
        locs.append(places_)
    places=pd.concat(locs,axis=0)
    for l in locs:
        del l
    #TODO check multiple appearence 
    places=places.groupby('TELJES_NEV').agg("last")
    places_with_gtfs=places.merge(gtfs_stops,how='left', left_on='POLGARI_NEV', right_on='stop_name')
    return places_with_gtfs


In [39]:
locations=get_location_data()

In [70]:
locations.head()

,OR_KOD,SZH_KOD,SZH_NEV,POLGARI_NEV,SZH_TIP_KOD,TEHERVONAT,UTASFORGALOM,UZEMVALT,HATARALLOMAS,UZEM_VVALL_VPE_ID,...,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,55,5504135,Aba-Sárkeresztúr,Aba-Sárkeresztúr,1,1.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,5507229,Abaliget,Abaliget,1,1.0,1.0,0,0,2.0,...,Abaliget,NaN,46.152778,18.073611,NaN,NaN,0.0,NaN,NaN,2.0
2,55,5515610,Abapuszta,Abapuszta,2,0.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0
4,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0


In [40]:
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
'''
# Coordinates of two locations
lat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland
lat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy

# Calculate the distance
distance = calculate_distance(lat1, lon1, lat2, lon2)
print(f"The distance is {distance} kilometers.")
'''

'\n# Coordinates of two locations\nlat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland\nlat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy\n\n# Calculate the distance\ndistance = calculate_distance(lat1, lon1, lat2, lon2)\nprint(f"The distance is {distance} kilometers.")\n'

In [41]:
weather_meta_file_name=weather_folder+"weather_meta_avg.csv"
met_stat_locations=pd.read_csv(weather_meta_file_name,sep=',',encoding='iso-8859-2')
met_stat_locations

,Loc,Lat,Lon
0,Budapest,47.507300,19.031400
1,Debrecen,47.531438,21.625387
2,Keszthely,46.742617,17.227400
3,Miskolc,48.123882,20.837100
4,Nyiregyhaza,47.969911,21.767344
5,Pecs,46.087188,18.277832
6,Sopron,47.687800,16.603233
7,Szeged,46.251943,20.114371
8,Szombathely,47.249480,16.623060
9,Turkeve,47.105900,20.734733


In [42]:
def get_min_dist_loc(loc_df,lat,lon):
    loc_df['d']=loc_df.apply(lambda x: calculate_distance(lat,lon,x['Lat'],x['Lon']),axis=1)
    return loc_df.loc[loc_df['d'].idxmin()]['Loc']
def get_Locations_coordinates(locations_df,met_locations):
    coord_dict=locations_df[['SZH_KOD','stop_lat','stop_lon','SZH_NEV','POLGARI_NEV','stop_name']].loc[locations_df['stop_lat'].notna()]
    coord_dict['SZH_KOD']=coord_dict['SZH_KOD'].apply(lambda x: str(x)[2:]).astype(int)
    coord_dict['Legközelebbi met. állomás']=coord_dict.apply(lambda x: get_min_dist_loc(met_locations,x['stop_lat'],x['stop_lon']),axis=1)
    return coord_dict
coord_dict=get_Locations_coordinates(locations,met_stat_locations)

In [43]:
coord_dict=coord_dict.drop_duplicates()

In [44]:
coord_dict.to_pickle(generated_files_path+"stat_coord_dict.pkl")

In [13]:
#a=events['IDO'].unique()

In [14]:
#days=pd.to_datetime(events['KOZLEKEDESI_NAP'],format='%Y.%m.%d',errors='coerce')

In [15]:
'''
min_date=days.min()
max_date=days.max()
print(min_date,' - ',max_date)
'''

"\nmin_date=days.min()\nmax_date=days.max()\nprint(min_date,' - ',max_date)\n"

Mit használunk a késésnél előrejelzésnél?
- Szakasz: 
- előző késés
- előző vonat áthaladás
- előző vonat késés 

- szakaszhoz hozzáadjuk pálya állapotot

potenciálisan :
- átlagsebesség a szakaszon



In [12]:
weather_file_name='weather.csv'
import datetime

weather_df=pd.read_csv(datafolder_base+weather_file_name,sep=',',encoding='iso-8859-2',parse_dates=['Time'],date_format='%Y-%m-%d')
#weather_df=weather_df.loc[weather_df['Time'].between(min_date,max_date)]


In [2]:
weather_df.tail()

NameError: name 'weather_df' is not defined

In [14]:
collected_trains={}
collected_trains[2]=[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2090, 2103, 2117, 2127, 2171, 2178, 2181, 2188, 2193, 2203, 2204, 2282, 2294, 3210, 3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3224, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3234, 3235, 3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257, 3258, 3259, 3260, 3261, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3280, 3281, 3282, 3283, 3284, 3285, 3286, 3287]
collected_trains[147]=[17243, 32610, 32612, 32613, 32614, 32615, 32616, 32617, 32619, 32626, 37410, 37412, 37414, 37415, 37416, 37417, 37418, 37419, 37421, 37428, 37431, 37435, 37436, 37438, 37439, 37452, 37454, 37455, 37456, 37457, 37461, 37463, 37465, 37466, 37467]

In [19]:
import pandas as pd
from config import delay_causes_dir
import os

files=os.listdir(delay_causes_dir)
dcr=[]
delay_category_tables=None
def process_delay_causes(train_table,train_event_table,reasoning_table, monitored_trains_array):
    reasons=reasoning_table[["VeID","VkOkSszam","KfkId","SkodId","IgPerc","Indoklas","Vonatszam","VonatDatum"]].copy()
    reasons.rename(columns={'Vonatszam':'OkozoVonat','VonatDatum':'OkozoVonatDatum'},inplace=True)
    train_events=train_event_table[["VeID","VonatID","SzhIndex","SzhKod","EsemID","EsemIdo","KesesPerc"]].copy()
    trains=train_table[['VonatID','vonatszam','vonatdatum']].copy()
    trains.rename(columns={'vonatszam': 'ErintettVonatSzam','vonatdatum':'ErintettVonatDatum'}, inplace=True)
    other_train_caused=reasons[(~reasons["OkozoVonat"].isnull()) & (reasons['KfkId'].isin([6,119]))]
    
    train_caused_reasons_usable=other_train_caused[other_train_caused["OkozoVonat"].isin(monitored_trains_array)]
    full_train_table=train_caused_reasons_usable.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']
    train_deps=full_train_table[['OkozoVonat','ErintettVonatSzam','SzhKod','OkozoErintettNapKulonbseg',"KfkId"]].drop_duplicates()
    
    dep_support=full_train_table[['OkozoVonat','ErintettVonatSzam','SzhKod','OkozoErintettNapKulonbseg',"KfkId"]].value_counts()
    
    other_reasons=reasons[~reasons['KfkId'].isin([6,119])]
    train_caused_reasons_unusable=other_train_caused[~other_train_caused["OkozoVonat"].isin(monitored_trains_array)]
    other_usable_reasons=other_reasons[other_reasons["KfkId"].isin([13,110,22,118,188])]
    other_unusable_reasons=other_reasons[~other_reasons["KfkId"].isin([13,110,22,118,188])]
    train_caused_reasons_unusable=train_caused_reasons_unusable.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    other_unusable_reasons=other_unusable_reasons.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    to_drop=pd.concat([train_caused_reasons_unusable,other_unusable_reasons],ignore_index=True)
    to_drop=to_drop[['OkozoVonatDatum','ErintettVonatDatum','ErintettVonatSzam']]
    
    return train_deps,dep_support,to_drop
    
#def get_train_deps(train_delays_table):
def get_delay_causes():
    deps={}    
    drops={}
    sups={}
    for f in files:
        if not f.startswith('Vonat'):
            if f.startswith('Torzs'):
                delay_category_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Esemeny(EsemID)', 'KesesiFokod(KfkID)','KesesiSegedkod(KskID)'])
            print("Ignoring file: {}".format(f))
            continue
        l=int(f.replace('.xlsx','').split("_")[-1])
        if not l in deps.keys():
            deps[l]=[]
            drops[l]=[]
            sups[l]=[]
        delay_cause_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Vonat', 'VonatEsemeny','KesesIndoklas'])
        dep_table,sup,to_drop=process_delay_causes(delay_cause_tables['Vonat'],delay_cause_tables['VonatEsemeny'],delay_cause_tables['KesesIndoklas'],collected_trains[l])
        deps[l].append(dep_table)
        drops[l].append(to_drop)
        sups[l].append(sup)
    dependencies={}
    drop_trains={}
    for l in deps.keys():
        dependencies[l]=pd.concat(deps[l],ignore_index=True)
    for l in drops.keys():
        drop_trains[l]=pd.concat(drops[l],ignore_index=True)
    return dependencies,sups,drop_trains

deps,sup,drops=get_delay_causes()

Ignoring file: Torzs.xlsx


C:\Users\ELTE\AppData\Local\Temp\ipykernel_2512\438237265.py:18: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']
C:\Users\ELTE\AppData\Local\Temp\ipykernel_2512\438237265.py:18: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']


Ignoring file: ~$Torzs.xlsx
Ignoring file: ~$Vonat_2022_2.xlsx
Ignoring file: ~$Vonat_2022_2023_147.xlsx


In [20]:
deps[2][deps[2]['OkozoErintettNapKulonbseg']!=pd.Timedelta(0)]

,OkozoVonat,ErintettVonatSzam,SzhKod,OkozoErintettNapKulonbseg,KfkId
227,2010.0,2181,10082,1 days 00:00:00,119
1115,3248.0,3238,10074,1 days 00:00:00,119
1777,2181.0,2188,1453,1 days 00:00:00,119
1859,2082.0,2077,10082,2 days 00:00:00,119
2000,2010.0,2181,10082,1 days 00:00:00,119
2010,2188.0,2181,1453,1 days 00:00:00,119
2143,2074.0,2065,10082,3 days 00:00:00,119
2258,2072.0,2067,10082,1 days 00:00:00,119
2309,2040.0,2019,10082,1 days 00:00:00,119
2496,2010.0,2181,10124,1 days 00:00:00,119


In [21]:
drops[2].shape

(31105, 3)

In [22]:
deps[2].shape

(3090, 5)

In [23]:
deps[2].head()

,OkozoVonat,ErintettVonatSzam,SzhKod,OkozoErintettNapKulonbseg,KfkId
0,2066.0,2053,10082,0 days 00:00:00,119
1,2066.0,2073,1453,0 days 00:00:00,119
2,3236.0,3233,10124,0 days 00:00:00,119
3,3246.0,3243,10124,0 days 00:00:00,119
4,3246.0,2083,40253,0 days 00:00:00,119


In [20]:
for k,v in deps.items():
    v.to_pickle(generated_files_path+f'train_dependencies_{k}.pkl')
for k,v in drops.items():
    v.to_pickle(generated_files_path+f'trains_to_drop_{k}.pkl')

# Egyetlen eseményfájl feldolgozása

Relatív késéshez:

Armstrong's original definition is as follows:

 $$\text{SMAPE} = \frac{1}{n} \sum_{t=1}^n \frac{\left|F_t-A_t\right|}{(A_t+F_t)/2}$$
 miket érdekel az eltérés iránya is  (Teny-Terv)/(Teny+Terv)

In [44]:
import pandas as pd
from config import event_file_name,passanger_info_folder,data_root
import numpy as np
import datetime


date_format='%Y.%m.%d %H:%M:%S'
drop_cols=['IND_CEL_JELZO','HATARESEM_JELZO','KOFE_IDO','KOFE_MODDAT']+["VSZAM_RENDSZER","EMIG_IDO","EMIG_IDO","EMIG_MODDAT","FOR_IDO","FOR_MODDAT",'ADATJELLEG','LEMONDAS','UTMODDAT',"FOR00_ESEMENY",'VONAL','VONTATO_TIPUS',"UIC","MEGALL_JELZO","UIC","MOZDONYAZONOSITO","EMIG_ESEMENY"] #+["ESEMENY_SORSZAM","ALLOMAS_SORSZAM","MEGALLO_SORSZAM"]


# TODO lehet évenként kezelni
# TODO csatlakozás éjjelni vonatnál!! talán nincs ilyen...
def process_events(events,deps,drops):
    

    events=events.drop(columns=drop_cols)
    events['KOZLEKEDESI_NAP']=pd.to_datetime(events['KOZLEKEDESI_NAP'],format="%Y.%m.%d",errors='coerce')
    # datetime konvertálás, ahol 00:00:00 sima to_datetime lehal.. 
    events['IDO_']=pd.to_datetime(events['IDO'],format=date_format,errors='coerce')
    events['IDO_'] = events['IDO_'].fillna(pd.to_datetime(events['IDO'] + ' 00:00:00'))
    
    events['TENY_IDO_']=pd.to_datetime(events['TENY_IDO'],format=date_format,errors='coerce')
    events['TENY_IDO_'] = events['TENY_IDO_'].fillna(pd.to_datetime(events['TENY_IDO'] + ' 00:00:00'))
    events.drop(columns=['IDO','TENY_IDO'],inplace=True)
    events.rename(columns={'IDO_':'IDO','TENY_IDO_':'TENY_IDO'},inplace=True)
    # áthaladó szolgálati helyek duplikálása
    condition = events["ESEMENY"] == 3
    events = pd.concat([events, events[condition].copy()], ignore_index=True)
    events = events.sort_values(by=['KOZLEKEDESI_NAP','VONATSZAM','ESEMENY_SORSZAM' ])

    # TODO sok helyen hiányzik, főleg rákosrendező, nem látok okot rá.. 51170 alkalommal összesen.
    events['TENY_IDO'].fillna(events['IDO'],inplace=True)
    
    events['OSSZ_KESES']=events['TENY_IDO']-events['IDO']   
    # indulásnál elozo_esemeny_ido,terv_idtartam,teny_idotartam,relativ, elozo szakasz terv idotart elozo szakasz keses, elozo szakasz relativ keses hianyzik keses hianyzik -össz késés ok

    events['SZH_KOD']=pd.to_numeric(events.apply(lambda x:'%s%s' % (x['ORSZAG_KOD'],x['ALLOMAS']),axis=1))
    
    #esemeny 4 valami olyasmi, hogy megjelnik a vonat/szolgálatba áll
    # 2 indulás
    # 1 érkezés
    # 3 áthaladás
    # 5 kiáll a szolgálatból
    #itt most 4,5 dobható? amikor indulunk, érdekes lehet milyen késéssel tesszük azt. A terv idő itt 0 (4-2 esemény)
    # +vannak olyan állomások, ahol a tervezett tartózkodás 0 
    
    
    events[['ELOZO_OSSZ_KESES','ELOZO_ALLOMAS','ELOZO_ESEMENY_IDO','ELOZO_SZH_KOD']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['OSSZ_KESES','ALLOMAS','IDO','SZH_KOD']].shift()

    events['ELOZO_OSSZ_KESES'].fillna(pd.Timedelta(0),inplace=True)
   # TODO az első megállóknál problémás. Ha IDO-vel feltöltjük, akkor 0 lesz a tervezett időtartam, arelatív késés meg végtelen mi lesz a SMAPE? lehet az lenne a járható út, ha a trainingre, evaluationre dobnánk - 113330 ilyen, avlósínűleg ennyi vonatunk is volt.. van 314 32-es sorszám: 2188 -éjfélen átmenő vonat
    events['ELOZO_ESEMENY_IDO'].fillna(events['IDO'],inplace=True)

    events['KESES']=events['OSSZ_KESES']-events['ELOZO_OSSZ_KESES'] 
    events['OSSZ_KESES (m)']=events['OSSZ_KESES']/np.timedelta64(1, 'm')
    events['TERV_IDOTARTAM (m)']= (events['IDO']-events['ELOZO_ESEMENY_IDO']).dt.seconds/60
    events['KESES (m)']=events['KESES']/np.timedelta64(1, 'm')

    events['TENY_IDOTARTAM (m)']=events['TERV_IDOTARTAM (m)']+events['KESES (m)']
    
    #realtív késés problematikus ha 0 a terv idő és a tényidő is akkor végtelen lesz. deakkor nem tévedtünk. ->0val töltjük
    #áthaladásnál a sorduplikácoó miatt 0/0 lesz =NaN
    events['RELATIV_KESES']=  (events['TENY_IDOTARTAM (m)']-events['TERV_IDOTARTAM (m)'])/(events['TERV_IDOTARTAM (m)']+events['TENY_IDOTARTAM (m)'])
    events['RELATIV_KESES']=events['RELATIV_KESES'].replace([np.inf,-np.inf,np.NaN],0.0)
    events[['ELOZO_SZAKASZ_TERV_IDOTARTAM (m)','ELOZO_SZAKASZ_KESES (m)','ELOZO_SZAKASZ_RELATIV_KESES']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['TERV_IDOTARTAM (m)','KESES (m)','RELATIV_KESES']].shift()
    events[['ELOZO_SZAKASZ_TERV_IDOTARTAM (m)','ELOZO_SZAKASZ_KESES (m)']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['TERV_IDOTARTAM (m)','KESES (m)']].shift()

    events['ALLOMAS'] = events['ALLOMAS'].astype(int)
    events['SZH_KOD'] = events['SZH_KOD'].astype(int)

    events['ELOZO_ALLOMAS'].fillna(events['ALLOMAS'],inplace=True)
    #TODO KELL EZ?
    events['ELOZO_SZH_KOD'].fillna(events['SZH_KOD'],inplace=True)

    events['ID']=  events['ELOZO_ALLOMAS'].astype(int).astype(str)+"-"+events['ALLOMAS'].astype(str)

    
    deps['ID']=deps['SzhKod'].astype(str)+'-'+deps['SzhKod'].astype(str)
    deps.rename(columns={'ErintettVonatSzam':"VONATSZAM","ErintettVonatDatum":"KOZLEKEDESI_NAP"},inplace=True)
    deps['OkozoVonat']=deps['OkozoVonat'].astype(int)
    events=events.merge(deps,on=['ID','VONATSZAM'],how='left')
    # beérés késése
    events_deps=events[["VONATSZAM","TENY_IDO","KOZLEKEDESI_NAP","ALLOMAS","ELOZO_ALLOMAS"]][(events['ALLOMAS']!=events['ELOZO_ALLOMAS'])]        
    events=events.merge(events_deps,left_on=['SzhKod','OkozoVonat','KOZLEKEDESI_NAP'],right_on=['ALLOMAS','VONATSZAM','KOZLEKEDESI_NAP'],how='left')
    #hogyan viszonyul a késés tenyidő a TERVEZETT induláshoz (IDO)?
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['TENY_IDO_y']-events['IDO']
    #negativ= nem kesett
    #nagyon negativ egy nappal elobbi TENY_IDO_y - adjunk hozza napokat amig nem nagyon negativa
    #pozitiv = kesett
    #nagyon pozitiv egy nappal keseobbi TENY_IDO_y -vonajunk le
    def fix_delay(difference:pd.Timedelta):
        while(difference<-pd.Timedelta(720,'m')):
            difference+=pd.Timedelta(1,'d')
        while(difference>pd.Timedelta(720,'m')):
            difference-=pd.Timedelta(1,'d')
        return difference
    #events['KORRIGÁLT']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].map(fix_delay)
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].map(fix_delay)
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].fillna(pd.Timedelta(0),inplace=True)
    #events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'] if events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']>pd.Timedelta(0) else pd.Timedelta(0)
    events['CSATL_MAX_KESES (m)']=events.groupby(['KOZLEKEDESI_NAP','VONATSZAM_x','ID'])['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].transform(max)
    events['CSATL_MAX_KESES (m)']=events['CSATL_MAX_KESES (m)'].dt.total_seconds()/60
    #állomásoknál késve érkezés de időben indulásra
    events['PUFFER']=0
    events['PUFFER']=events.loc[events["ESEMENY"] == 2,'PUFFER']=events['TERV_IDOTARTAM (m)']-events['ELOZO_SZAKASZ_KESES (m)']

    
    #itt van egy kis probléma.. lehet hogy a csatlakozó vonat közlekedési napja más...
    #˘1440 perc körüli késés...
    # HACK: mondjuk 1000 felett akkor -1440? 
    #tehát más közlekedési napú vonatra várunk - közl nap + vonatid a kér vonatdatum nem fog egyezni a keses ok táblákban
    
    drops.rename(columns={'ErintettVonatDatum':'KOZLEKEDESI_NAP','ErintettVonatSzam':'VONATSZAM'},inplace=True)
    #TODO miért kell most?
    drops['KOZLEKEDESI_NAP']=pd.to_datetime(drops['KOZLEKEDESI_NAP'])
    drops.rename(columns={"VONATSZAM":"VONATSZAM_x"},inplace=True)
    events=events.merge(drops.drop_duplicates(), on=['KOZLEKEDESI_NAP','VONATSZAM_x'], 
                   how='left', indicator=True)
    print("Before drop: ",events.shape)
    events=events[events['_merge'] == 'left_only']
    events.drop(columns=['_merge'],inplace=True)
    
    print("After drop: ",events.shape)
    
    rename_after_merges={"VONATSZAM_x":"VONATSZAM","TENY_IDO_x":"TENY_IDO","ALLOMAS_x":"ALLOMAS"}

    delete_after_merges=["ALLOMAS_y","TENY_IDO_y","OkozoVonat",'ALLOMAS_y','TENY_IDO_y','ELOZO_ALLOMAS_y','CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST','KfkId','SzhKod','VONATSZAM_y',"ORSZAG_KOD","ELOZO_ALLOMAS_x","ELOZO_SZH_KOD","OkozoErintettNapKulonbseg","OkozoVonatDatum","OkozoErintettNapKulonbseg"] 
    events.rename(columns=rename_after_merges,inplace=True)
    events.drop(columns=delete_after_merges,inplace=True)
    events.drop_duplicates(inplace=True)
    

    #itt fontos lehet a dátum is, ezért érdemes az időt is megtartani
    # viszont az idő inkább a napszak miatt érdekes
    events['MENETREND_IDO (m)']=events['IDO'].dt.hour*60+events['IDO'].dt.minute
    #events['MENETREND_IDO (m)']=events['MENETREND_IDO (m)'].dt.total_seconds()/60
    #events=events[events['ESEMENY']!=4]
    events.drop(columns=['ELOZO_OSSZ_KESES','KESES','ELOZO_OSSZ_KESES','OSSZ_KESES']+["ALLOMAS_SORSZAM","MEGALLO_SORSZAM"],inplace=True)
    events=events.fillna(0.0)
    

    min_date=events['KOZLEKEDESI_NAP'].min()
    max_date=events['KOZLEKEDESI_NAP'].max()
    print(min_date,' - ',max_date)
    return events,min_date,max_date

def add_weather_data(coord_dict,weather_df,events):
    start_stations=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP']).first()['ALLOMAS'].reset_index()
    events=events.merge(start_stations,on=['KOZLEKEDESI_NAP','VONATSZAM'], how='left')
    events.rename(columns={'ALLOMAS_x':'ALLOMAS','ALLOMAS_y':'IndulóÁllomás'},inplace=True)
    #events['IndulóÁllomás']=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP']).first().reset_index()['ALLOMAS'].astype(int)
    events=events.merge(coord_dict, how='left', left_on='IndulóÁllomás',right_on='SZH_KOD')
    events = events.merge(weather_df, how='left', left_on=['Legközelebbi met. állomás', 'KOZLEKEDESI_NAP'],
                              right_on=['Loc', 'Time'])
    return events

def get_events_data(weather_df,coord_dict,deps,drops,year=2023,line=147 ):
    l= passanger_info_folder + str(year) + "/"
    e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
    
    e,min_date,max_date=process_events(e,deps[line],drops[line])
    weather_df1=weather_df.loc[weather_df['Time'].between(min_date,max_date)]

    e=add_weather_data(coord_dict,weather_df1,e)
    junk_from_joins=['Loc','SZH_KOD_x','SZH_KOD_y','stop_lat','stop_lon','Time','Legközelebbi met. állomás','IndulóÁllomás','ALLOMAS']
    
    
    e.drop(columns=junk_from_joins,inplace=True)
    e['LINE']=line
    return e
###############################################################################x
### FEJLESZTÉSHEZ #####
###############################################################################x

def load_deps_and_drops(lines):
    deps={}
    drops={}
    for l in lines:
        deps[l]=pd.read_pickle(data_root+f'train_dependencies_{l}.pkl')
        drops[l]=pd.read_pickle(data_root+f'trains_to_drop_{l}.pkl')
    return deps, drops

year=2023
line=147


deps,drops=load_deps_and_drops([147,2])
deps_=deps[line]
drops_=drops[line]


#l= passanger_info_folder + str(year) + "/"
# események file beolvasása, és feldolgozása a dependenciák és vonatkizárásokkal 
#e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
#res=process_events(e,deps_,drops_)
################################################################################ ##

# időjárás hozzáadásához szükséges generált file-ok betölrése
from config import generated_files_path
'''
weather_file_name='weather.csv'

datafolder_base = generated_files_path+"/odp/"

weather_df=pd.read_csv(datafolder_base+weather_file_name,sep=',',encoding='iso-8859-2',parse_dates=['Time'],date_format='%Y-%m-%d')
coord_dict=pd.read_pickle(generated_files_path+"stat_coord_dict.pkl")
'''
################################################################################ ##
# teljes feldolgozás: események, dependenciák és kizárt vonatok beolvasása, alap adat összerakása, valamint időjárás  beolvasása és alapadathoz adása
#events_2=get_events_data(weather_df,coord_dict,deps, drops,year,line)  



C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:159: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['IDO_'] = events['IDO_'].fillna(pd.to_datetime(events['IDO'] + ' 00:00:00'))
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['TENY_IDO_'] = events['TENY_IDO_'].fillna(pd.to_datetime(events['TENY_IDO'] + ' 00:00:00'))
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.

Before drop:  (243533, 39)
After drop:  (112862, 38)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:139: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2023-01-01 00:00:00  -  2023-12-31 00:00:00


In [9]:
weather_df.tail()

,Loc,Time,tx,t,tn,r
449245,Turkeve,2023-12-27,11.3,6.1,2.5,0.0
449246,Turkeve,2023-12-28,10.8,4.8,0.4,0.0
449247,Turkeve,2023-12-29,10.9,3.7,-0.4,0.0
449248,Turkeve,2023-12-30,10.7,3.8,-0.1,0.0
449249,Turkeve,2023-12-31,11.2,5.1,0.2,0.0


In [40]:
missing_met=events_2[pd.isnull(events_2['IndulóÁllomás'])]

In [41]:
missing_met.shape

(0, 35)

In [45]:
events_2.tail()

,VONATSZAM,KOZLEKEDESI_NAP,ESEMENY,ALLOMAS,ESEMENY_SORSZAM,IDO,TENY_IDO,SZH_KOD_x,ELOZO_ESEMENY_IDO,OSSZ_KESES (m),...,POLGARI_NEV,stop_name,Legközelebbi met. állomás,Loc,Time,tx,t,tn,r,LINE
107889,37467,2023-12-31,2,44933,15,2023-12-31 08:09:00,2023-12-31 08:09:44,5544933,2023-12-31 08:09:00,0.733333,...,Orosháza,Orosháza,Szeged,Szeged,2023-12-31,11.8,4.3,-1.4,0.0,147
107890,37467,2023-12-31,1,44925,16,2023-12-31 08:14:00,2023-12-31 08:14:16,5544925,2023-12-31 08:09:00,0.266667,...,Orosháza,Orosháza,Szeged,Szeged,2023-12-31,11.8,4.3,-1.4,0.0,147
107891,37467,2023-12-31,2,44925,17,2023-12-31 08:14:00,2023-12-31 08:14:34,5544925,2023-12-31 08:14:00,0.566667,...,Orosháza,Orosháza,Szeged,Szeged,2023-12-31,11.8,4.3,-1.4,0.0,147
107892,37467,2023-12-31,1,18184,18,2023-12-31 08:25:00,2023-12-31 08:25:44,5518184,2023-12-31 08:14:00,0.733333,...,Orosháza,Orosháza,Szeged,Szeged,2023-12-31,11.8,4.3,-1.4,0.0,147
107893,37467,2023-12-31,5,18184,19,2023-12-31 08:25:00,2023-12-31 08:25:00,5518184,2023-12-31 08:25:00,0.000000,...,Orosháza,Orosháza,Szeged,Szeged,2023-12-31,11.8,4.3,-1.4,0.0,147


In [46]:
del events_2

# Állatorvosi ló : 2188 2-es vonalon

In [26]:
#TODO nézzünk rá csatlakozás késésére is...
# TODO van még 0-ás -áthaladás? igen de nem ott!  
# TODO 40824 tokodi elégazás  áthaladás -3 as tervezett időtartamot kapunk rá (+1440-nincs k2-ben), pedig nem kellene vagy 0-val - leduplikálni 0-ás tervezett idővel? elcsúszik a 0ás tervezett idő!! az előző állomásnál jelenik meg
# simán töröljük  a megállót? nem kellene, mert akkor nem tudjuk vonalállapotra illeszteni... De ha meg benn marad figyelni élő predikciónál
#TODO NAT - éjfélnél előző időpont pont éjfél az időpont?? - NAT nem megy a parseolás -> ok
# TODO sima duplikálás beveszünk egy 0-ás tervezett idejűmegállót, aztán kitöröljük?



#to_check=res[0][(res[0]['VONATSZAM']==2188) & (pd.to_datetime(res[0]['KOZLEKEDESI_NAP'])==datetime.datetime(2022,12,11))]

KeyError: 0

In [27]:
#to_check

# indulásnál elozo_esemeny_ido,terv_idtartam,teny_idotartam,relativ, elozo szakasz terv idotart elozo szakasz keses, elozo szakasz relativ keses hianyzik keses hianyzik -össz késés ok

# van egy csomo 0 tervezett ido - allomasokon is dobjuk ki az ilyen tartózkodásokat is? már előre, késések számolása előtt?
# csak hackeljük meg pár másodpercre?

#simán kihagyom a relatív késést?, a megállás késésként fog megjelenni. 

NameError: name 'to_check' is not defined

In [19]:
#del res, to_check

In [39]:
import numpy as np
#elegánsabb lenne VONATSZAM és VONAL nélkül, de ahhot több adat kell majd..
# ELOZO_ESEMENY_IDO a line status joinhoz -terv
#TODO  LINE egy közelítés az állapot leírására, ezen majd lehet dolgozni azért...
#TODO ELOZO_KESES elkodolhat maradek információkat ezen is lehet majd dolgozni, bár rendenkezésre állhat
#TODO VONATSZAM szintén menetrendi, prioritás információk, de túl sok lehetőség,és ellentmond a logikánknak inkább érdemes dobni
#érdekes lehet : MOZDONYAZONOSITO, UIC, VONTATO_TIPUS
#to_keep=['VONAL','VONATSZAM','KOZLEKEDESI_NAP','VONTATO_TIPUS','IDO','ELOZO_ESEMENY_IDO','MENETREND_IDO','OSSZ_KESES (m)','ELOZO_SZAKASZ_KESES (m)','KESES (m)','RELATIV_KESES','TERV_IDOTARTAM (m)','CSATL_MAX_KESES','tx','t','tn','r']
#def get_events_pred_data(processed, line=2):
    #ezek azok amik kellenek most, és később is
    #processed=processed[to_keep]
    #processed['LINE']=line
    #return processed
#d2=get_events_pred_data(events_2,147)
#d2.to_pickle(data_root+'test_events_data.pkl')


# Vonatfej - TODO
nem tudom mire lesz még jó..

# Menetrend variációk Vizualizációja

### Data Preprocessing 

# Lassújel és vágányzár

In [28]:
import pandas as pd

slow_signal_tables = pd.read_excel('data/obstacles/lassújel_2_147_vonal_20220101-20231231_AI-hoz.xlsx',
                                   sheet_name=['ZVZ_KEIHEAD', 'ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])
closure_tables = pd.read_excel('data/obstacles/vágányzár_2_147_vonal_20220101-20231231_AI-hoz.xlsx    ',
                               sheet_name=['ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])


In [29]:
ss_parts=slow_signal_tables['ZVZ_KIZARASOK'].copy()
ss_parts.drop(axis=0,index=0,inplace=True)


ss_parts['Kizárásszám']=ss_parts['Kizárásszám'].astype(str)


to_drop=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]
#todo később jól jöhetnek: 
# Lassúmenet kezdő szelvény, Lassúmenet záró szelvény,Lassúmenet mértéke tehervonatra (km/h),Kis teljesítményű dízel kitoló (csupa 0),Dízel kitoló szükséges (nincs ilyen),Nagy teljesítményű dízel kitoló (nincs ilyen),Állomási bizt.ber. kikapcsolás szükséges(nincs ilyen)
# kérdéses: Változó munkahelyen kitűzött, Kizárás típusa (redundáns?)
#todo kezdő, záró szolgálati hely mindig egy minimális szakasz?
cols= ss_parts.columns
for d in to_drop:
    if d not in cols:
        print('Missing:',d)
    
ss_parts.drop(columns=to_drop,inplace=True)

#lassúmenet iránya egy adott sínpárra/vágányra vonatkozik, de sínpár is lehet
# Vágányzár VZ írja le melyik vágányról beszélünk.

# ha > akkor a jobb oldali lehet érdekesebb 
#ss_parts['S_ID']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Kezdő szolgálati hely'],x['Záró szolgálati hely']) if ">" in x["Lassúmenet iránya"] else -1   ,axis=1)

# ha  < akkor a bal 
#ss_parts['S_ID_R']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Záró szolgálati hely'],x['Kezdő szolgálati hely']) if "<" in x["Lassúmenet iránya"] else -1 ,axis=1)

# kellene olyasmi hogy Irány szerinti oldal, és ellentétes oldal lezárva ehhez érdemes lehet kettészedni az irányokat..
# todo: VZ van jobb/bal oldali közlekedés? most abból indulunk ki, hogy van valmilyen szinten azaz szakszmenti és ellentétes - J: I(rány) a nem fordítottnál, E(llentétes) a fordítottnál, B  E(llenttes) a nem fordítottnál és I a fordítottnál

#todo VZ elméletben lehet A is, gyakorlatban csak 1 és 4 van
ss_parts.loc[ss_parts['Záró szolgálati hely'].isna(),'Záró szolgálati hely']=ss_parts['Kezdő szolgálati hely']

ss_parts_fw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains(">")]
ss_parts_bw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains("<")]

def set_section_id(df):
    df['Kezdő szolgálati hely']=df['Kezdő szolgálati hely'].str.replace('M-55','')
    df['Záró szolgálati hely']=df['Záró szolgálati hely'].str.replace('M-55','')
    
    df['ID']=df['Kezdő szolgálati hely'].astype(str)+"-"+df['Záró szolgálati hely'].astype(str)

def encode_directions(dff,dfb):
    dff["I"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dff["E"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dff["ÁI"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dff["ÁE"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    
    dfb["I"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dfb["E"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dfb["ÁI"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    dfb["ÁE"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dfb[['Kezdő szolgálati hely','Záró szolgálati hely']]=dfb[['Záró szolgálati hely','Kezdő szolgálati hely']]
    return pd.concat([dff,dfb],axis=0)


ss_parts=encode_directions(ss_parts_fw,ss_parts_bw)
set_section_id(ss_parts)

ss_parts = ss_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Lassúmenet iránya','Vágány (VZ)','Átmenő vágány típusa']) 

ss_parts.head()

,Kizárásszám,Érintett objektum típusa,Lassúmenet típusa,Lassúmenet hossza,Lassúmenet mértéke (km/h),Lassúmenet mértéke motorvonatra (km/h),Fels.vez. kikapcsolása,I,E,ÁI,ÁE,ID
1,100000103765,0,I,100,80.0,NaN,NaN,1,1,0,0,01503-01511
2,100000104230,0,I,100,20.0,NaN,NaN,1,1,0,0,18184-17475
3,100000104231,0,I,100,20.0,NaN,NaN,1,1,0,0,18184-17475
4,100000104553,0,I,300,20.0,NaN,NaN,1,1,0,0,10207-40253
5,100000104555,0,I,300,20.0,NaN,NaN,1,1,0,0,10207-40253


In [30]:
# csak lassújelnél
# helyettesítsük 0-val VZ-nál?
ss_parts['Lassúmenet mértéke motorvonatra (km/h)']=ss_parts['Lassúmenet mértéke motorvonatra (km/h)'].fillna(ss_parts['Lassúmenet mértéke (km/h)'])
#todo pontosítás lassúmenet mekkora része az adott szakasznak? - A tervidő segíthet
#todo milyen vonat?? motor? 

In [31]:
import numpy as np
# igen/nem
ss_parts['Fels.vez. kikapcsolása']=ss_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})
# ideiglenes az, ami hatással lehet a késésre, elvileg az állandónak már benne kellene lenni a menetrendben 
ss_parts['Lassúmenet típusa']=ss_parts['Lassúmenet típusa'].map({'Á':0,'I':1})
#ss_parts=pd.get_dummies(ss_parts, columns=['Érintett objektum típusa'],dtype=int)


In [32]:
ss_status = slow_signal_tables['ZVZ_KEIHEAD'].copy()

ss_status.drop(axis=0,index=0,inplace=True)
ss_status['Vágányzár KEI ID']=ss_status['Vágányzár KEI ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
ss_status['Kezdés']= pd.to_datetime(ss_status['Utolsó jóváhagyott kezdés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott kezdés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
ss_status['Befejezés']= pd.to_datetime(ss_status['Utolsó jóváhagyott befejezés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott befejezés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')


'''
ss_status['Tény-indítás (dátum)']=pd.to_datetime( ss_status['Tény-indítás (dátum)'],format='%Y-%m-%d').dt.date   
ss_status['TényIndít.időpontja']=pd.to_datetime( ss_status['TényIndít.időpontja']).dt.time
ss_status['Tény-befejez.(dátum)']=pd.to_datetime( ss_status['Tény-befejez.(dátum)'],format='%Y-%m-%d').dt.date   
ss_status['Tény-befej.ideje']=pd.to_datetime( ss_status['Tény-befej.ideje']).dt.time
'''

to_drop_head=['Státusséma','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Státusséma','Vágányzár Státus','Korlátozás típus']
ss_status=ss_status.drop(columns=to_drop_head)


In [33]:
merged=ss_parts.merge(ss_status,how='left',left_on='Kizárásszám',right_on='Vágányzár KEI ID')


In [34]:
c_head=closure_tables['ZVZ_VZTHEAD'].copy()
# "lemondott" -  akkor ez nem történt meg?
canceled=['KNJM','LEMD','LEMM','LEMO','KELU']
# "törölt" ez mit jelent ? szintén nem történt meg?   
deleted=['ELUT','VPEE','VTÖR','KELU']
not_valid=canceled+deleted
c_head['Státus'] = 1
c_head.loc[c_head['Vágányzár Státus'].isin(not_valid), "Státus"] = 0

c_head['Kiutalt menetvonalak zavartatása nélkül'] = c_head['Kiutalt menetvonalak zavartatása nélkül'].map(
    {np.nan: 0, 'X': 1})
c_head.drop(axis=0,index=0,inplace=True)
c_head['Vágányzár VZT ID']=c_head['Vágányzár VZT ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
c_head['Kezdés']= pd.to_datetime(c_head['Tény-indítás (dátum)'].astype(str)+' '+c_head['TényIndít.időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
c_head['Befejezés']= pd.to_datetime(c_head['Tény-befejez.(dátum)'].astype(str)+' '+c_head['Tény-befej.ideje'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ementrend szempontjából nem érdekes
to_drop_c_head=['Korlátozás típus','Korlátozottan.rend.álló kap.','Státusséma','Vágányzár Státus','Tény-indítás (dátum)','TényIndít.időpontja','Tény-befejez.(dátum)','Tény-befej.ideje','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Vágányzár túllépve','Van vonatkésés','Pótolni kell']

c_head.drop(columns=to_drop_c_head,inplace=True)

In [35]:
c_parts = closure_tables['ZVZ_KIZARASOK'].copy()

c_parts.loc[c_parts['Záró szolgálati hely'].isna(), 'Záró szolgálati hely'] = c_parts['Kezdő szolgálati hely']
c_parts_bw=c_parts.copy()



#TODO  Vágány (VZ) és 'Átmenő vágány típusa' ugyanannak néz ki mint lassújelnél, ugyanazt a logikát csináljuk itt is 
#itt vannak 1,2,3,4 értékek VZ-nél, elvileg lehet 'A' itt is 

c_parts=encode_directions(c_parts,c_parts_bw)
set_section_id(c_parts)

c_parts = c_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Vágány (VZ)','Átmenő vágány típusa']) 
to_drop_gen=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]

to_drop_ss=['Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)']
c_parts['Fels.vez. kikapcsolása']=c_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})

c_parts.drop(columns=to_drop_gen,inplace=True)

#TODO ezen azért lehet majd még gondolkodni...
#c_parts['Lassúmenet mértéke (km/h)']=0.0
#c_parts['Lassúmenet mértéke motorvonatra (km/h)']=0.0
#TODO ez lehet gáz
#c_parts['Lassúmenet hossza']=0
#c_parts['Lassúmenet típusa']=1

c_parts['Kizárásszám']=c_parts['Kizárásszám'].astype(str).str.replace('.0','')

c_merged=c_parts.merge(c_head,how='left',left_on='Kizárásszám',right_on='Vágányzár VZT ID')



In [36]:
merged['Kizárás (1=VZ,2=LJ)']=2
c_merged['Kizárás (1=VZ,2=LJ)']=1

In [37]:
line_status=pd.concat([merged,c_merged])
del merged,c_merged
line_status.head()

,Kizárásszám,Érintett objektum típusa,Lassúmenet típusa,Lassúmenet hossza,Lassúmenet mértéke (km/h),Lassúmenet mértéke motorvonatra (km/h),Fels.vez. kikapcsolása,I,E,ÁI,ÁE,ID,Vágányzár KEI ID,Kezdés,Befejezés,"Kizárás (1=VZ,2=LJ)",Vágányzár VZT ID,Kiutalt menetvonalak zavartatása nélkül,Státus
0,100000103765,0,1.0,100.0,80.0,80.0,0,1,1,0,0,01503-01511,100000103765,2022-04-12 15:25:15,2022-10-13 10:00:00,2,NaN,NaN,NaN
1,100000104230,0,1.0,100.0,20.0,20.0,0,1,1,0,0,18184-17475,100000104230,2022-04-25 04:30:00,2022-04-25 20:00:00,2,NaN,NaN,NaN
2,100000104231,0,1.0,100.0,20.0,20.0,0,1,1,0,0,18184-17475,100000104231,2022-04-26 04:30:00,2022-04-26 14:00:00,2,NaN,NaN,NaN
3,100000104553,0,1.0,300.0,20.0,20.0,0,1,1,0,0,10207-40253,100000104553,2022-05-02 08:00:00,2022-05-02 15:00:00,2,NaN,NaN,NaN
4,100000104555,0,1.0,300.0,20.0,20.0,0,1,1,0,0,10207-40253,100000104555,2022-05-03 08:00:00,2022-05-03 15:00:00,2,NaN,NaN,NaN


In [38]:
final_drop=['Vágányzár KEI ID','Vágányzár VZT ID']
line_status.drop(columns=final_drop,inplace=True)

In [39]:
line_status.fillna(0.0,inplace=True)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_2512\263976322.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  line_status.fillna(0.0,inplace=True)


In [40]:
line_status.head()

,Kizárásszám,Érintett objektum típusa,Lassúmenet típusa,Lassúmenet hossza,Lassúmenet mértéke (km/h),Lassúmenet mértéke motorvonatra (km/h),Fels.vez. kikapcsolása,I,E,ÁI,ÁE,ID,Kezdés,Befejezés,"Kizárás (1=VZ,2=LJ)",Kiutalt menetvonalak zavartatása nélkül,Státus
0,100000103765,0,1.0,100.0,80.0,80.0,0,1,1,0,0,01503-01511,2022-04-12 15:25:15,2022-10-13 10:00:00,2,0.0,0.0
1,100000104230,0,1.0,100.0,20.0,20.0,0,1,1,0,0,18184-17475,2022-04-25 04:30:00,2022-04-25 20:00:00,2,0.0,0.0
2,100000104231,0,1.0,100.0,20.0,20.0,0,1,1,0,0,18184-17475,2022-04-26 04:30:00,2022-04-26 14:00:00,2,0.0,0.0
3,100000104553,0,1.0,300.0,20.0,20.0,0,1,1,0,0,10207-40253,2022-05-02 08:00:00,2022-05-02 15:00:00,2,0.0,0.0
4,100000104555,0,1.0,300.0,20.0,20.0,0,1,1,0,0,10207-40253,2022-05-03 08:00:00,2022-05-03 15:00:00,2,0.0,0.0


In [41]:
from config import generated_files_path
fn=generated_files_path+'line_status.csv'
line_status.to_pickle(fn)
del line_status

# load
df = pd.read_pickle(fn)

In [ ]:
df.head()

# Végső összerakás

In [47]:
import pandas as pd
def add_line_info(e, line_info):
    #e=events.copy()
    #d2.reset_index(inplace=True)
    joined = e.merge(line_info,on='ID',how='inner')
    # todo miért kell???
    joined['Kezdés']=pd.to_datetime(joined['Kezdés'])
    joined['Befejezés']=pd.to_datetime(joined['Befejezés'])
    #ezek azok az események, ahol volt valami korlátozás
    joined = joined[joined['ELOZO_ESEMENY_IDO'].between(joined['Kezdés'],joined['Befejezés'])]
    e=e.reindex(columns=joined.columns)
    e.update(joined)
    e.fillna(0.0,inplace=True)
    e.drop(columns='ELOZO_ESEMENY_IDO',inplace=True)
    return e



fn=generated_files_path+'line_status.csv'
line_status = pd.read_pickle(fn)
#events_with_line_info=add_line_info(d2, line_status)
#events_with_line_info.head()
    

In [43]:
line_status['ID'].unique()

array(['01503-01511', '18184-17475', '10207-40253', '17418-17434',
       '01420-01438', '47456-17434', '10082-10124', '17475-18614',
       '10124-10124', '10124-10207', '17434-17434', '17434-18184',
       '01495-01495', '01511-01511', '01503-01503', '17475-17475',
       '17418-17418', '17137-42895', '47456-17418', '01487-01487',
       '01404-01404', '01404-01420', '01440-01442', '01442-01453',
       '01453-01453', '01453-01487', '01438-01438', '01438-01440',
       '01420-01420', '18184-42879', '40824-01503', '42879-42879',
       '01440-01440', '01442-01442', '42895-42895', '42895-47456',
       '47456-47456', '42879-17475', '10207-10207', '01511-01503',
       '17475-18184', '40253-10207', '17434-17418', '01438-01420',
       '17434-47456', '10124-10082', '18614-17475', '10207-10124',
       '18184-17434', '42895-17137', '17418-47456', '01420-01404',
       '01442-01440', '01453-01442', '01487-01453', '01440-01438',
       '42879-18184', '01503-40824', '47456-42895', '17475-428

In [48]:
def add_previous_train_delay(e):
    #e=events.copy()
    #TODO érdemes lehet mozdonyid-ra is csoportosítani
    e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']]=e.groupby('ID')[['KESES (m)','Kizárásszám','RELATIV_KESES']].shift()
    e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']].fillna(0.0,inplace=True)
    e['VONAL_STATUSZ_VALTOZOTT']=e['ELOZO_STATUS']!=e['Kizárásszám']
    e.drop(columns=['ELOZO_STATUS','Kizárásszám'],inplace=True)
    return e

#events_with_line_info=add_previous_train_delay(events_with_line_info)

In [49]:
#deps,drops=get_delay_causes()
from config import weather_folder,data_root
import datetime

lines=[2,147]

deps,drops=load_deps_and_drops(lines)
weather_file_name='weather.csv'

weather_df=pd.read_csv(weather_folder+weather_file_name,sep=',',encoding='iso-8859-2',parse_dates=['Time'],date_format='%Y-%m-%d')
coord_dict=pd.read_pickle(data_root+"stat_coord_dict.pkl")


In [50]:
from config import years_to_process
events=[] 
for year in years_to_process:
    for line in [2,147]:
        #esemény és időjárás adat
        e = get_events_data(weather_df,coord_dict,deps,drops,year,line)
        e = add_line_info(e,line_status)
        e = add_previous_train_delay(e)
        e.fillna(0.0,inplace=True)
        events.append(e)
data=pd.concat(events)

C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:159: DtypeWarning: Columns (16,17,28) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['IDO_'] = events['IDO_'].fillna(pd.to_datetime(events['IDO'] + ' 00:00:00'))
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['TENY_IDO_'] = events['TENY_IDO_'].fillna(pd.to_datetime(events['TENY_IDO'] + ' 00:00:00'))
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\27234

Before drop:  (4380630, 39)
After drop:  (3158739, 38)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:139: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2022-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10033 10082 10082 ... 40121 40121 10074]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5510033 5510082 5510082 ... 5540121 5540121 5510074]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 25  29  30 ... 747 747 749]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined

...

C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\3622305359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']].fillna(0.0,inplace=True)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:159: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['IDO_'] = events['IDO_'].fillna(pd.to_datetime(events['IDO'] + ' 00:00:00'))
C:\Users\ELTE\Ap

Before drop:  (243533, 39)
After drop:  (112862, 38)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\2723457686.py:139: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2023-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[18572 18572 18564 ... 44925 18184 18184]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5518572 5518572 5518564 ... 5544925 5518184 5518184]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_18520\4268978556.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[946 946 950 ... 494 505 505]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined

In [ ]:
# TODO nem az igazi, pkl helyette most
#save_data_with_types(data,data_root+'data.csv')

    

In [51]:
data.fillna(0.0,inplace=True)

In [52]:
data.to_pickle(generated_files_path+'data.pkl')

In [53]:
data.head(30)

,VONATSZAM,KOZLEKEDESI_NAP,ESEMENY,ALLOMAS,ESEMENY_SORSZAM,IDO,TENY_IDO,SZH_KOD_x,OSSZ_KESES (m),TERV_IDOTARTAM (m),...,ÁI,ÁE,Kezdés,Befejezés,"Kizárás (1=VZ,2=LJ)",Kiutalt menetvonalak zavartatása nélkül,Státus,ELOZO_VONAT_KESES (m),ELOZO_VONAT_RELATIV_KESES,VONAL_STATUSZ_VALTOZOTT
0,2010,2022-01-01,2,10033,1,2022-01-01 00:25:00,2022-01-01 00:25:24,5510033,0.400000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,2010,2022-01-01,1,10082,2,2022-01-01 00:29:00,2022-01-01 00:30:10,5510082,1.166667,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,2010,2022-01-01,2,10082,3,2022-01-01 00:30:00,2022-01-01 00:30:40,5510082,0.666667,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,2010,2022-01-01,1,10124,4,2022-01-01 00:35:00,2022-01-01 00:34:55,5510124,-0.083333,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,2010,2022-01-01,2,10124,5,2022-01-01 00:36:00,2022-01-01 00:36:09,5510124,0.150000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
5,2010,2022-01-01,1,10207,6,2022-01-01 00:38:00,2022-01-01 00:37:44,5510207,-0.266667,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
6,2010,2022-01-01,2,10207,7,2022-01-01 00:38:00,2022-01-01 00:38:17,5510207,0.283333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
7,2010,2022-01-01,1,40253,8,2022-01-01 00:41:00,2022-01-01 00:40:58,5540253,-0.033333,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
8,2010,2022-01-01,2,40253,9,2022-01-01 00:42:00,2022-01-01 00:42:12,5540253,0.200000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
9,2010,2022-01-01,1,1404,10,2022-01-01 00:43:00,2022-01-01 00:43:44,551404,0.733333,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [54]:
del data


# Késési ok meta

In [54]:
dc={}
reasonings=[]
for f in files:
    if not f.startswith('Vonat'):
        if f.startswith('Torzs'):
            delay_category_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Esemeny(EsemID)', 'KesesiFokod(KfkID)','KesesiSegedkod(KskID)'])
        print("Ignoring file: {}".format(f))
        continue
    l=f.replace('.xlsx','').split("_")[-1]
    if not l in dc.keys():
        dc[l]=[]
    delay_cause_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Vonat', 'VonatEsemeny','KesesIndoklas'])
    dc[l].append(delay_cause_tables)
for line,l in dc.items():
    for dfs in l:
        delay_causes=dfs['KesesIndoklas'].copy()
        delay_causes['Line']=line
        reasonings.append(delay_causes)
reasonings_df=pd.concat(reasonings, ignore_index=True)



Ignoring file: Torzs.xlsx
Ignoring file: ~$Torzs.xlsx
Ignoring file: ~$Vonat_2022_2.xlsx
Ignoring file: ~$Vonat_2022_2023_147.xlsx


In [55]:
reasonings_df.head()

,VeID,VkOkSszam,KfkId,SkodId,IgPerc,Indoklas,Vonatszam,VonatDatum,RDEAzon,Line
0,188313858,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
1,188314407,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
2,188314939,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
3,188316339,1,22,36.0,3,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
4,188316340,1,13,29.0,1,utasforgalom,NaN,NaT,NaN,2


In [56]:
train_causes=reasonings_df[~reasonings_df['Vonatszam'].isnull()]
print(train_causes.shape)
explainable=train_causes[train_causes['Vonatszam'].isin(collected_trains[147]+collected_trains[2])].copy()
print(explainable.shape)
not_explainable=train_causes[~train_causes['Vonatszam'].isin(collected_trains[147]+collected_trains[2])].copy()
print(not_explainable.shape)



(68590, 10)
(52080, 10)
(16510, 10)


In [ ]:
explainable.head()

In [ ]:
reasonings_df.shape

In [58]:
delay_category_tables['KesesiFokod(KfkID)'][['kfk_id','kfk_megnevezes']]

,kfk_id,kfk_megnevezes
0,1,Forgalmi előkészítés
1,2,Vonatforgalom szabályozás megrendelői igények ...
2,3,Vonatforgalomszabályozás menetrendtől eltérő k...
3,4,Helytelen forgalomszabályozás
4,5,Forgalmi okból hívójelzés
...,...,...
186,188,Sebességkorlátozás külön rendelet alapján
187,189,Nem közölt
188,190,Közlekedő vonat balesete
189,191,Kerekítés lokális menetidőhiány miatt


In [59]:
delay_categories=reasonings_df['KfkId'].value_counts()
delay_categories

KfkId
119    42318
110    29091
7      16851
22     16754
13     11153
6       8581
123     3439
118     3354
145     1550
2        756
142      712
135      411
133      310
129      282
188      255
136      168
156      125
171      124
19       121
134       96
154       87
151       74
155       58
172       51
126       45
153       39
138       35
150       34
169       27
140       20
173       13
161        6
175        6
130        5
127        4
181        4
174        2
148        2
122        2
191        2
132        1
103        1
Name: count, dtype: int64

In [60]:
delay_stat_df=pd.DataFrame(delay_categories).merge(delay_category_tables['KesesiFokod(KfkID)'][['kfk_id','kfk_megnevezes']],left_on='KfkId',right_on='kfk_id',how='left')

In [61]:
print(delay_stat_df.to_latex())


\begin{tabular}{lrrl}
\toprule
 & count & kfk_id & kfk_megnevezes \\
\midrule
0 & 42318 & 119 & Vonatforgalom szabályozás más vonat késése miatt \\
1 & 29091 & 110 & Menetidő túllépés \\
2 & 16851 & 7 & Fordulásra várás \\
3 & 16754 & 22 & Ideiglenes sebességkorlátozás \\
4 & 11153 & 13 & Tartózkodási idő túllépése \\
5 & 8581 & 6 & Csatlakozásra várás \\
6 & 3439 & 123 & Vasúttársasági előkészítés \\
7 & 3354 & 118 & Vonatforgalom szabályozás a vonat korábbi (saját) késése miatt \\
8 & 1550 & 145 & Vonatszemélyzeti okok \\
9 & 756 & 2 & Vonatforgalom szabályozás megrendelői igények alapján \\
10 & 712 & 142 & Forgalmi személyzeti okok  \\
11 & 411 & 135 & Látszólagos foglaltság \\
12 & 310 & 133 & Biztosítóberendezési zavar \\
13 & 282 & 129 & Vontatójármű zavarai \\
14 & 255 & 188 & Sebességkorlátozás külön rendelet alapján \\
15 & 168 & 136 & Útátjáró biztosítóberendezési zavar \\
16 & 125 & 156 & Idegen behatások \\
17 & 124 & 171 & Tervtől eltérő vonatösszeállítás vasútvállalati o

Csinálunk egy dependency table-t ahol megjelenik, melyik vonatra vár az adott vonatunk, hol. A várt vonat aktuális(?) késését aztán az adott szakaszon (állomáson) bevesszük az attribútumok közé.

Ahol nincs információnk az adott vonatról a historikus adatokban, azokat a vonatokat eldobjuk.


# Generált fileok

# összes vonat menetrendje

In [2]:
from utils import get_train_data
from config import not_existing,generated_files_path,collected_trains
import time

rets = {}
i = 0
for v in collected_trains.values():
    for no in v:
        if no in not_existing:
            print("Seem to be nonexisting..")
        #if i>2:
        #    continue
        i += 1
        print('pulling ', no)
        rets[no] = get_train_data(no)
        time.sleep(5)
import pickle

pickle.dump(rets, open(generated_files_path+'full_train_pull_20_05.pkl', 'wb'))

pulling  2010
-> Pulling  2010
pulling  2011
-> Pulling  2011
pulling  2012
-> Pulling  2012
pulling  2013
-> Pulling  2013
pulling  2014
-> Pulling  2014
pulling  2015
-> Pulling  2015
pulling  2016
-> Pulling  2016
pulling  2017
-> Pulling  2017
pulling  2018
-> Pulling  2018
pulling  2019
-> Pulling  2019
pulling  2020
-> Pulling  2020
----------------------------- Failed 2020
pulling  2021
-> Pulling  2021
pulling  2022
-> Pulling  2022
pulling  2023
-> Pulling  2023
pulling  2024
-> Pulling  2024
pulling  2025
-> Pulling  2025
pulling  2026
-> Pulling  2026
pulling  2027
-> Pulling  2027
pulling  2028
-> Pulling  2028
pulling  2029
-> Pulling  2029
pulling  2030
-> Pulling  2030
pulling  2031
-> Pulling  2031
pulling  2032
-> Pulling  2032
pulling  2033
-> Pulling  2033
pulling  2034
-> Pulling  2034
pulling  2035
-> Pulling  2035
pulling  2036
-> Pulling  2036
pulling  2037
-> Pulling  2037
pulling  2038
-> Pulling  2038
pulling  2039
-> Pulling  2039
pulling  2040
-> Pulling  20

In [15]:
list(rets.values())[0].head()

,Km,Állomás,Vágány,ERK_TERV,ERK_TENY,IND_TERV,IND_TENY,VONAT,KOZLEKEDESI_NAP,NEV,TiPUS,PLUSZ
0,0,Budapest-Nyugati,2,NaT,NaT,2024-12-13 00:25:00,2024-12-13 00:24:00,2010,2024-12-13,S72,személyvonat,
1,3,Rákosrendező,3,2024-12-13 00:29:00,2024-12-13 00:31:00,2024-12-13 00:30:00,2024-12-13 00:31:00,2010,2024-12-13,S72,személyvonat,
2,7,Angyalföld,2,2024-12-13 00:35:00,2024-12-13 00:35:00,2024-12-13 00:36:00,2024-12-13 00:36:00,2010,2024-12-13,S72,személyvonat,
3,8,Újpest,2,2024-12-13 00:38:00,2024-12-13 00:38:00,2024-12-13 00:38:00,2024-12-13 00:38:00,2010,2024-12-13,S72,személyvonat,
4,10,Aquincum,1,2024-12-13 00:41:00,2024-12-13 00:41:00,2024-12-13 00:42:00,2024-12-13 00:42:00,2010,2024-12-13,S72,személyvonat,


# GTFS automatizált letöltése


In [ ]:
from datetime import datetime
import os
import dotenv
import shutil
import requests


from utils import download_gtfs_zip





download_gtfs_zip()
import zipfile
from config import gtfs_dowload_location

extracted_folder = gtfs_dowload_location + "latest/gtfs"

print(f'Extracting {gtfs_dowload_location + "latest/gtfs"} ')
# create folder with the same name of zip, and extract content in it
with zipfile.ZipFile(gtfs_dowload_location + "latest/gtfs.zip", 'r') as zip_ref:
    if not os.path.exists(extracted_folder):
        os.mkdir(extracted_folder)
    zip_ref.extractall(extracted_folder)
    # iterate over all the downloaded files (per zip, with hourly update we expect a single file)
    for f in os.listdir(extracted_folder):
        print(f)

## járatok geometriája

- a routes-ból összelegózzuk a nevet
    - a route állhat több tripből.. 
    - ahol név alapján joinolunk- ott nincs meg a konkrét vonat duplikát pathok lehetnek de a leghosszabb jó kell legyen mindenhova
    - ahol van szám: únió?
    - leghosszabb route kell, de egy route lehet több trip is? route-oknál tripes uniója, aztán leghosszabb route?
- a trips-ből a shape-id-t
- a shapes-ből a geometriát..

In [1]:
import pandas as pd
from config import gtfs_dowload_location
routes=pd.read_csv(gtfs_dowload_location + "latest/gtfs/routes.txt")

In [2]:
routes.head()


,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,100019.0,198,NaN,Sz 9640,NaN,2,NaN,NaN,NaN
1,10002.0,198,NaN,IC 850 TÓPART,NaN,2,NaN,NaN,NaN
2,100020.0,198,NaN,Sz 9636,NaN,2,NaN,NaN,NaN
3,100021.0,198,NaN,B 9640,NaN,3,https://www.mavcsoport.hu/mav-start/belfoldi-u...,ffff00,000000
4,100021.1,198,NaN,Sz 9640,NaN,2,NaN,NaN,NaN


In [3]:
routes['numbers'] = routes["route_long_name"].map(lambda long_name: ' '.join([ i for i in str(long_name).strip().split(" ") if i.isnumeric()]))


In [4]:

#route short name -
# számok és ic nevek
num_routes = routes[routes['numbers'] != 'nan']
routes['name'] = ""
routes.loc[~routes['route_long_name'].isnull(), 'name'] = routes['route_long_name']
routes.loc[~routes['route_short_name'].isnull(), 'name'] = routes['route_short_name']
#itt megtartjuk a lehetőséget hogy több route tartozik egy vonathoz
routes = routes[['route_id', 'name','numbers']].drop_duplicates()





In [5]:
routes.head()


,route_id,name,numbers
0,100019.0,Sz 9640,9640
1,10002.0,IC 850 TÓPART,850
2,100020.0,Sz 9636,9636
3,100021.0,B 9640,9640
4,100021.1,Sz 9640,9640


In [6]:
routes[['route_id_main','section']]=routes['route_id'].astype(str).str.split('.',expand=True)


In [7]:
routes.head()

,route_id,name,numbers,route_id_main,section
0,100019.0,Sz 9640,9640,100019,0
1,10002.0,IC 850 TÓPART,850,10002,0
2,100020.0,Sz 9636,9636,100020,0
3,100021.0,B 9640,9640,100021,0
4,100021.1,Sz 9640,9640,100021,1


In [8]:
trips = pd.read_csv(gtfs_dowload_location + "latest/gtfs/trips.txt")

trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,110489.0,2616,14873108_2616.,NaN,NaN,NaN,NaN,14873108.0,NaN,NaN
1,110489.0,2619,14873108_2619.,NaN,NaN,NaN,NaN,14873108.0,NaN,NaN
2,4931.0,2616,14873122_2616.,NaN,NaN,NaN,NaN,14873122.0,NaN,NaN
3,4931.0,2619,14873122_2619.,NaN,NaN,NaN,NaN,14873122.0,NaN,NaN
4,50400.0,2191,14891343_2191.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
trips[['route_id_main','section']]=trips['route_id'].astype(str).str.split('.',expand=True)


In [10]:
trips.groupby('route_id')['trip_id'].count()

route_id
1009.0      2
1010.0      2
1036.0      2
1037.0      2
1038.0      2
           ..
111455.0    2
111464.0    1
111469.0    6
111470.0    2
111471.0    1
Name: trip_id, Length: 11069, dtype: int64

In [11]:
trips.loc[trips['shape_id'].isna(),'shape_id'] = trips['trip_id'].apply(lambda x:float(x.split('_')[0]) )
#megtartjuk, hogy a route hoz több trip is tartozhat
trips = trips[['route_id','route_id_main','section','service_id','trip_id','shape_id']].drop_duplicates()
trips.head()


,route_id,route_id_main,section,service_id,trip_id,shape_id
0,110489.0,110489,0,2616,14873108_2616.,14873108.0
1,110489.0,110489,0,2619,14873108_2619.,14873108.0
2,4931.0,4931,0,2616,14873122_2616.,14873122.0
3,4931.0,4931,0,2619,14873122_2619.,14873122.0
4,50400.0,50400,0,2191,14891343_2191.,14891343.0


In [12]:
print(routes.shape)
shapes = routes.merge(trips)
print(shapes.shape)

(11069, 5)
(38191, 8)


In [13]:
shapes.head()

,route_id,name,numbers,route_id_main,section,service_id,trip_id,shape_id
0,100019.0,Sz 9640,9640,100019,0,2001,19729712_2001.,19729712.0
1,100019.0,Sz 9640,9640,100019,0,2012,19729712_2012.,19729712.0
2,100019.0,Sz 9640,9640,100019,0,2283,19729712_2283.,19729712.0
3,100019.0,Sz 9640,9640,100019,0,2616,19729712_2616.,19729712.0
4,10002.0,IC 850 TÓPART,850,10002,0,2616,17502485_2616.,17502485.0


In [14]:
shapes.groupby(['name','route_id'])['shape_id'].value_counts()

name     route_id  shape_id  
1        55574.0   15250200.0    5
         55904.0   15261421.0    5
         55905.0   15261422.0    5
         55986.0   15250196.0    5
         55987.0   15250197.0    5
                                ..
rjx 67   69861.0   21112563.0    3
         69862.0   21112565.0    2
         69917.0   21112564.0    1
rjx 68   21112.0   19659620.0    2
         21200.0   21343998.0    2
Name: count, Length: 11069, dtype: int64

array(['9640', '9636', '6619', '6629', '9649', '9643', '9641', '9020',
       '19018', '19014', '201', '959', '955', '963', '957', '967', '953',
       '951', '9866', '9860', '969', '918', '904', '906', '908', '914',
       '900', '902', '916', '204', '956', '954', '966', '950', '958',
       '964', '952', '312', '190', '192', '182', '5008', '984', '986',
       '982', '992', '996', '988', '1998', '994', '6088', '6098', '6008',
       '9600', '8950', '9009', '19609', '9633', '19027', '19023', '6009',
       '6019', '6646', '6628', '16338', '6318', '6328', '6338', '6310',
       '', '6638', '6648', '8218', '8208', '37038', '6311', '6621',
       '6643', '313', '8219', '8229', '19695', '19697', '19693', '19603',
       '19607', '19601', '19605', '9910', '9930', '9932', '9918', '9928',
       '9958', '9920', '9912', '9922', '9914', '9924', '9934', '9926',
       '9916', '9936', '9946', '9938', '19938', '9948', '8117', '8113',
       '8115', '1320', '1324', '1322', '1306', '1308', '1310', 

In [16]:
import pickle
from config import not_existing,generated_files_path
#TODO ezt kicsit dinamikusabban kellene, egy beégetett fájl jelenleg...
file = open(generated_files_path+'full_train_pull_20_05.pkl', 'rb')

rets = pickle.load(file)
naming_data_from_api={}
for k,v in rets.items():
    print(k)
    '''
    if k in not_existing:
        print('not existing ',k)
        continue
    '''
    if v is None :
        print(f'missing train: {k} ')
        not_existing+=[k]
        continue
    naming_data_from_api[k]={} 
    name=v['NEV'].loc[0]
    type=v['TiPUS'].loc[0]
    if type.isupper():
        print(type)
        t=name
        name=type
        type=t
    if name.islower():
        t=name
        name=type
        type=t
    naming_data_from_api[k]['NEV']=name
    naming_data_from_api[k]['TIPUS']=type
    naming_data_from_api[k]['VONATSZAM']=k


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
missing train: 2020 
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
missing train: 2069 
2070
missing train: 2070 
2071
2072
2073
2074
2075
2076
2077
2078
2080
missing train: 2080 
2081
2082
2083
2084
2085
2086
2087
2088
2090
missing train: 2090 
2103
missing train: 2103 
2117
2127
2171
2178
2181
2188
2193
missing train: 2193 
2203
missing train: 2203 
2204
missing train: 2204 
2282
missing train: 2282 
2294
missing train: 2294 
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
missing train: 3250 
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
missing train: 3260 
3261
3262
3263
3264
3265
32

In [17]:
mapping = pd.DataFrame(naming_data_from_api.values())

mapping
#merged_df = mapping.merge(shapes, how='inner', left_on='NEV', right_on='name', suffixes=('_left', '_right'))


,NEV,TIPUS,VONATSZAM
0,S72,személyvonat,2010
1,Z72,személyvonat,2011
2,Z72,személyvonat,2012
3,Z72,személyvonat,2013
4,Z72,személyvonat,2014
...,...,...,...
168,,személyvonat,37431
169,,személyvonat,37435
170,,személyvonat,37438
171,,személyvonat,37439


In [312]:
train_nos=mapping.VONATSZAM.unique()

In [313]:
train_nos

array([ 2010,  2011,  2012,  2013,  2014,  2015,  2016,  2017,  2018,
        2019,  2021,  2022,  2023,  2024,  2025,  2026,  2027,  2028,
        2029,  2030,  2031,  2032,  2033,  2034,  2035,  2036,  2037,
        2038,  2039,  2040,  2041,  2042,  2043,  2044,  2045,  2046,
        2047,  2048,  2049,  2050,  2051,  2052,  2053,  2054,  2055,
        2056,  2057,  2058,  2060,  2061,  2062,  2063,  2064,  2065,
        2066,  2067,  2068,  2071,  2072,  2073,  2074,  2075,  2076,
        2077,  2078,  2081,  2082,  2083,  2084,  2085,  2086,  2087,
        2088,  2117,  2127,  2171,  2178,  2181,  2188,  3210,  3211,
        3212,  3213,  3214,  3215,  3216,  3217,  3218,  3219,  3220,
        3221,  3222,  3223,  3224,  3226,  3227,  3228,  3229,  3230,
        3231,  3232,  3233,  3234,  3235,  3236,  3237,  3238,  3239,
        3240,  3241,  3242,  3243,  3244,  3245,  3246,  3247,  3248,
        3249,  3251,  3252,  3253,  3254,  3255,  3256,  3257,  3258,
        3259,  3261,

In [18]:
mapping.loc[mapping['NEV']=="",'NEV']=mapping['VONATSZAM'].astype(str)

In [19]:
mapping['NEV'].unique()

array(['S72', 'Z72', 'S76', '32610', '32612', '32613', '32614', '32615',
       '32616', '32617', '32619', '32626', '37410', '37412', '37414',
       '37415', '37416', '37417', '37418', '37419', '37421', '37428',
       '37431', '37435', '37438', '37439', '37461'], dtype=object)

In [20]:
shapes_with_nums=shapes.loc[shapes['numbers']!=""]
shapes_with_names=shapes.loc[shapes['numbers']==""]

In [21]:
#itt van egy erős tippünk a vonatszámra:

In [22]:
shapes_with_nums.head(100)

,route_id,name,numbers,route_id_main,section,service_id,trip_id,shape_id
0,100019.0,Sz 9640,9640,100019,0,2001,19729712_2001.,19729712.0
1,100019.0,Sz 9640,9640,100019,0,2012,19729712_2012.,19729712.0
2,100019.0,Sz 9640,9640,100019,0,2283,19729712_2283.,19729712.0
3,100019.0,Sz 9640,9640,100019,0,2616,19729712_2616.,19729712.0
4,10002.0,IC 850 TÓPART,850,10002,0,2616,17502485_2616.,17502485.0
...,...,...,...,...,...,...,...,...
505,101477.0,9422,9422,101477,0,2448,20839231_2448.,20839231.0
506,101477.0,9422,9422,101477,0,2449,20839231_2449.,20839231.0
507,101477.0,9422,9422,101477,0,2542,20839231_2542.,20839231.0
508,101477.0,9422,9422,101477,0,2543,20839231_2543.,20839231.0


In [23]:
shapes_with_nums['VONATSZAM']=shapes_with_nums['numbers'].astype(int)

C:\Users\ELTE\AppData\Local\Temp\ipykernel_4952\1985058361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shapes_with_nums['VONATSZAM']=shapes_with_nums['numbers'].astype(int)


In [24]:
complicated_route_train_no_s=shapes_with_nums.loc[shapes_with_nums['section']!='0']['VONATSZAM'].unique()


In [25]:
train_nos=mapping.VONATSZAM.unique()


In [26]:
import numpy as np
intersection = np.intersect1d(train_nos, complicated_route_train_no_s)

In [27]:
intersection

array([], dtype=int64)

In [28]:


m1=mapping.merge(shapes_with_nums)

In [326]:
m1=m1[['NEV','VONATSZAM','route_id','route_id_main','section','shape_id']].drop_duplicates()

In [327]:
m1['NEV'].unique()

array(['32610', '32612', '32613', '32614', '32615', '32616', '32617',
       '32619', '32626', '37410', '37412', '37414', '37415', '37416',
       '37417', '37418', '37419', '37421', '37428', '37431', '37435',
       '37438', '37439', '37461'], dtype=object)

In [30]:
print(m1.groupby(['VONATSZAM','route_id'])['shape_id'].value_counts())

VONATSZAM  route_id  shape_id  
32610      59228.0   19120859.0     5
           59233.0   20840303.0    12
32612      59223.0   15360443.0     2
           59229.0   20838975.0     2
32613      56242.0   20842582.0    12
           56244.0   15263115.0     5
32614      59206.0   20840633.0     2
           59225.0   15361050.0     2
32615      56236.0   15361057.0     2
           56265.0   20839028.0     2
32616      59224.0   15360445.0     2
           59230.0   20838982.0     4
32617      56235.0   15361056.0     2
           56241.0   20840659.0     2
32619      56238.0   19120740.0     2
           56240.0   20840304.0     4
32626      59226.0   15361054.0     1
           59231.0   20838987.0     9
37410      90865.0   20838632.0    12
           90867.0   15261511.0     5
37412      59577.0   15360438.0     2
           59593.0   20840681.0     2
37414      59581.0   15361040.0     2
           59590.0   20839045.0     2
37415      90846.0   15361042.0     2
           90859.0

In [31]:
shapes_with_names['NEV']=shapes_with_names['name']


C:\Users\ELTE\AppData\Local\Temp\ipykernel_4952\1336091705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shapes_with_names['NEV']=shapes_with_names['name']


In [32]:
shapes_with_names1=shapes_with_names[['route_id','route_id_main','section','NEV','shape_id']].drop_duplicates()


In [33]:
shapes_with_names1.head(50)

,route_id,route_id_main,section,NEV,shape_id
16,100028.0,100028,0,S30,20851113.0
30,100042.0,100042,0,G44,17502260.0
32,100043.0,100043,0,S30,18038802.0
37,100045.0,100045,0,G44,20839594.0
39,100055.0,100055,0,G44,20839629.0
51,100060.0,100060,0,G44,16751915.0
98,100485.0,100485,0,S36,20492200.0
129,100954.0,100954,0,S36,20297984.0
130,100955.0,100955,0,S36,20492201.0
131,100956.0,100956,0,S36,20842957.0


In [34]:
m2=mapping.merge(shapes_with_names1)


In [35]:
m2=m2[['NEV','VONATSZAM','route_id','route_id_main','section','shape_id']].drop_duplicates()

In [38]:
print(m2.groupby(['VONATSZAM','route_id'])['shape_id'].value_counts())

VONATSZAM  route_id  shape_id  
2010       27939.0   20838731.0    1
           27940.0   20840316.0    1
           27964.0   20179146.0    1
           27965.0   20577872.0    1
           27966.0   20577874.0    1
                                  ..
3287       78816.0   19918975.0    1
           78817.0   19918977.0    1
           78818.0   19918979.0    1
           79012.0   19922500.0    1
           79013.0   20840200.0    1
Name: count, Length: 28354, dtype: int64


In [39]:
m1.head()

,NEV,TIPUS,VONATSZAM,route_id,name,numbers,route_id_main,section,service_id,trip_id,shape_id
0,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2002,19120859_2002.,19120859.0
1,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2394,19120859_2394.,19120859.0
2,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2395,19120859_2395.,19120859.0
3,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2396,19120859_2396.,19120859.0
4,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2618,19120859_2618.,19120859.0


In [40]:
m2.head()

,NEV,VONATSZAM,route_id,route_id_main,section,shape_id
0,S72,2010,27939.0,27939,0,20838731.0
1,S72,2010,27940.0,27940,0,20840316.0
2,S72,2010,27964.0,27964,0,20179146.0
3,S72,2010,27965.0,27965,0,20577872.0
4,S72,2010,27966.0,27966,0,20577874.0


In [41]:
m=pd.concat([m1,m2])

In [42]:
#full join:  mindent mindennel, hogy kb név összehasonlítást tudjunk csinálni
#merged_df = mapping.assign(key=1).merge(shapes.assign(key=1), on='key')
#merged_df.head()

In [15]:
# akkor rendelünk egy gtfs vonatot az API-ből jövő vonathoz, ha GTFS-ből összelegózott név tartalmazza az API-ból jövő nevet
#merged_df['contains'] = merged_df.apply(lambda row: row['NEV'] in row['name'], axis=1)


In [16]:
#merged_df.head()

,NEV,TIPUS,VONATSZAM,key,route_id,name,shape_id,contains
0,S72,személyvonat,2010,1,100019.0,Sz 9640,19729712.0,False
1,S72,személyvonat,2010,1,10002.0,IC 850 TÓPART,17502485.0,False
2,S72,személyvonat,2010,1,100020.0,Sz 9636,19729713.0,False
3,S72,személyvonat,2010,1,100021.0,B 9640,19729819.0,False
4,S72,személyvonat,2010,1,100021.1,Sz 9640,19729819.0,False


In [ ]:
#itt most van egy csomó rout_id-nk minden vonathoz

In [43]:
# csak azokat tartjuk meg ahol volt név match
#merged_df = merged_df[merged_df['contains']]
#merged_df = merged_df[merged_df['NEV'] != '']
#merged_df.tail()
#merged_df_all_route_ids=merged_df.drop_duplicates()
m.to_pickle(generated_files_path + 'gtfs_shapes_full.pkl')

#final = merged_df.groupby("VONATSZAM").first().reset_index()
#from config import data_root

#final.to_pickle(generated_files_path + 'gtfs_shapes.pkl')

In [44]:
del m1, m2, shapes_with_names, shapes_with_names1, trips, routes,shapes_with_nums

Ezen a ponton megvannak a vonat_id-k, nevek, az összes előforduló route_id- és shap_id-vel. Ebből kellene kivanni az egész vonalat.. Ez a feltehetőleg route_id a legtöbb különböző ponttal.. 

# Shapes 

In [45]:
import pickle
import pandas as pd
from config import generated_files_path,gtfs_dowload_location

file = open(generated_files_path+'gtfs_shapes_full.pkl', 'rb')
mapping = pickle.load(file)
#több shape_id egy rout id-hoz. meg kellene találni a leghosszabbat a az egész vonalhoz..
shapes=pd.read_csv(gtfs_dowload_location + "latest/gtfs/shapes.txt")
mapping_with_shapes=mapping.merge(shapes, how='left')
mapping_with_shapes=mapping_with_shapes.groupby('VONATSZAM')['shape_pt_sequence'].first().reset_index()

In [46]:
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,14873108,46.837506,16.848204,4,NaN
1,14873108,46.838111,16.848584,5,NaN
2,14873108,46.838787,16.849305,6,NaN
3,14873108,46.841029,16.852747,7,NaN
4,14873108,46.841802,16.853749,8,NaN


In [47]:
mapping.head()

,NEV,TIPUS,VONATSZAM,route_id,name,numbers,route_id_main,section,service_id,trip_id,shape_id
0,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2002.0,19120859_2002.,19120859.0
1,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2394.0,19120859_2394.,19120859.0
2,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2395.0,19120859_2395.,19120859.0
3,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2396.0,19120859_2396.,19120859.0
4,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2618.0,19120859_2618.,19120859.0


In [114]:
# minden egyes geometriai pont minden vonathoz, azaz az összes rout_id-hez tartozó pontok .

In [52]:
mapping_with_shapes.groupby("VONATSZAM")["route_id"].value_counts()

VONATSZAM  route_id
2010       27939.0      41
           27940.0      41
           27964.0      41
           27965.0      41
           27966.0      41
                      ... 
37438      90879.0     120
37439      73260.0      48
           73252.0      20
37461      73265.0      48
           73247.0      20
Name: count, Length: 28402, dtype: int64

In [275]:
mapping_with_shapes[['VONATSZAM','route_id','shape_id']].drop_duplicates()

,VONATSZAM,route_id,shape_id
0,32610,59228.0,19120859.0
26,32610,59233.0,20840303.0
52,32612,59223.0,15360443.0
78,32612,59229.0,20838975.0
104,32613,56242.0,20842582.0
...,...,...,...
491474,3287,78816.0,19918975.0
491475,3287,78817.0,19918977.0
491476,3287,78818.0,19918979.0
491477,3287,79012.0,19922500.0


In [116]:
#1:1 route_id : shape_id, 1 vonatszam több routid
#meg kell találni shape-t/rout-ot a leghosszabb úttal

In [48]:
a=mapping_with_shapes.groupby(['VONATSZAM','shape_id'])['shape_pt_sequence'].nunique().reset_index()


In [56]:
a.index

RangeIndex(start=0, stop=28402, step=1)

In [57]:
a.loc[a['VONATSZAM']==37418]


,VONATSZAM,shape_id,shape_pt_sequence
28384,37418,15360439.0,23
28385,37418,20839046.0,23


In [59]:
max_indices = a.groupby('VONATSZAM')['shape_pt_sequence'].idxmax()
max_indices

VONATSZAM
2010         3
2011        21
2012       207
2013       393
2014       579
         ...  
37431    28392
37435    28394
37438    28396
37439    28398
37461    28400
Name: shape_pt_sequence, Length: 173, dtype: int64

In [60]:
max_values = a.loc[max_indices]

In [61]:
max_values=max_values.sort_values(by="VONATSZAM")

In [62]:
max_values.tail(30)

,VONATSZAM,shape_id,shape_pt_sequence
27074,3282,19918926.0,17
27288,3283,19918926.0,17
27502,3284,19918926.0,17
27716,3285,19918926.0,17
27930,3286,19918926.0,17
28144,3287,19918926.0,17
28354,32610,19120859.0,26
28356,32612,15360443.0,26
28358,32613,15263115.0,40
28360,32614,15361050.0,26


In [63]:
mapping.head()

,NEV,TIPUS,VONATSZAM,route_id,name,numbers,route_id_main,section,service_id,trip_id,shape_id
0,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2002.0,19120859_2002.,19120859.0
1,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2394.0,19120859_2394.,19120859.0
2,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2395.0,19120859_2395.,19120859.0
3,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2396.0,19120859_2396.,19120859.0
4,32610,személyvonat,32610,59228.0,VESz 32610,32610,59228,0,2618.0,19120859_2618.,19120859.0


In [64]:
single_shape_ids_with_train_ids=max_values.merge(mapping,how='left')

In [65]:
single_shape_ids_with_train_ids.sort_values(by="VONATSZAM").head()

,VONATSZAM,shape_id,shape_pt_sequence,NEV,TIPUS,route_id,name,numbers,route_id_main,section,service_id,trip_id
0,2010,17502326.0,41,S72,NaN,28067.0,NaN,NaN,28067,0,NaN,NaN
1,2011,16698407.0,41,Z72,NaN,28190.0,NaN,NaN,28190,0,NaN,NaN
2,2012,16698407.0,41,Z72,NaN,28190.0,NaN,NaN,28190,0,NaN,NaN
3,2013,16698407.0,41,Z72,NaN,28190.0,NaN,NaN,28190,0,NaN,NaN
4,2014,16698407.0,41,Z72,NaN,28190.0,NaN,NaN,28190,0,NaN,NaN


In [66]:
train_desc_with_geom_ids=single_shape_ids_with_train_ids.merge(shapes,  how='left')

In [67]:
train_desc_with_geom_ids['shape_id']=train_desc_with_geom_ids['shape_id'].astype(int)
train_desc_with_geom_ids=train_desc_with_geom_ids[['NEV','VONATSZAM','shape_id','route_id']]

In [68]:
train_desc_with_geom_ids.head()

,NEV,VONATSZAM,shape_id,route_id
0,S72,2010,17502326,28067.0
1,Z72,2011,16698407,28190.0
2,Z72,2012,16698407,28190.0
3,Z72,2013,16698407,28190.0
4,Z72,2014,16698407,28190.0


In [69]:
shapes=pd.read_csv(gtfs_dowload_location + "latest/gtfs/shapes.txt")


In [70]:
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,14873108,46.837506,16.848204,4,NaN
1,14873108,46.838111,16.848584,5,NaN
2,14873108,46.838787,16.849305,6,NaN
3,14873108,46.841029,16.852747,7,NaN
4,14873108,46.841802,16.853749,8,NaN


In [71]:
train_with_shapes=train_desc_with_geom_ids.merge(shapes, how='left')
train_with_shapes=train_with_shapes.sort_values(by=['VONATSZAM','shape_pt_sequence'])
train_with_shapes.head(50)

#TODO miért jött vissza az összes shape és route_id?

,NEV,VONATSZAM,shape_id,route_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,S72,2010,17502326,28067.0,47.513937,19.062601,2,NaN
1,S72,2010,17502326,28067.0,47.521402,19.081125,3,NaN
2,S72,2010,17502326,28067.0,47.523667,19.086589,4,NaN
3,S72,2010,17502326,28067.0,47.524138,19.087455,5,NaN
4,S72,2010,17502326,28067.0,47.527296,19.090996,8,NaN
5,S72,2010,17502326,28067.0,47.538040,19.099935,9,NaN
6,S72,2010,17502326,28067.0,47.540759,19.101780,10,NaN
7,S72,2010,17502326,28067.0,47.541436,19.101920,11,NaN
8,S72,2010,17502326,28067.0,47.608803,18.914315,86,NaN
9,S72,2010,17502326,28067.0,47.618662,18.896665,101,NaN


In [73]:
train_with_shapes.loc[train_with_shapes['VONATSZAM']==37418].drop_duplicates()

,NEV,VONATSZAM,shape_id,route_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
5546,37418,37418,15360439,59578.0,46.685751,19.975598,17,NaN
5547,37418,37418,15360439,59578.0,46.686951,19.993145,18,NaN
5548,37418,37418,15360439,59578.0,46.691141,20.047841,20,NaN
5549,37418,37418,15360439,59578.0,46.691510,20.049282,21,NaN
5550,37418,37418,15360439,59578.0,46.695157,20.059647,22,NaN
5551,37418,37418,15360439,59578.0,46.699931,20.074299,23,NaN
5552,37418,37418,15360439,59578.0,46.700705,20.077055,25,NaN
5553,37418,37418,15360439,59578.0,46.709179,20.103056,26,NaN
5554,37418,37418,15360439,59578.0,46.709351,20.104121,27,NaN
5555,37418,37418,15360439,59578.0,46.711339,20.124188,28,NaN


In [109]:

#todo nem kellene hogy legyenek duplikátumok
mapping_with_shapes=train_with_shapes[['VONATSZAM','shape_pt_lat','shape_pt_lon','shape_pt_sequence']].drop_duplicates()


In [110]:
mapping_with_shapes.head(50)

,VONATSZAM,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,2010,47.513937,19.062601,2
1,2010,47.521402,19.081125,3
2,2010,47.523667,19.086589,4
3,2010,47.524138,19.087455,5
4,2010,47.527296,19.090996,8
5,2010,47.538040,19.099935,9
6,2010,47.540759,19.101780,10
7,2010,47.541436,19.101920,11
8,2010,47.608803,18.914315,86
9,2010,47.618662,18.896665,101
